In [1]:
import nltk

# from nltk.parse.stanford import StanfordDependencyParser
# path_to_jar = '/Development/Projects/Magnifis/3rd_party/NLU/stanford-corenlp-full-2013/stanford-corenlp-3.2.0.jar'
# path_to_models_jar ='/Development/Projects/Magnifis/3rd_party/NLU/stanford-corenlp-full-2013/stanford-corenlp-3.2.0-models.jar'
# dependency_parser = StanfordDependencyParser(path_to_jar=path_to_jar, path_to_models_jar=path_to_models_jar)

# result = dependency_parser.raw_parse('I shot an elephant in my sleep')
# dep = result.next()
# list(dep.triples())


In [2]:
import os
import io
raw_corpus = ''
dir_name = './similes_train/'   
fList=os.listdir(dir_name)
fList1 = [os.path.join(dir_name, f) for f in fList if os.path.isfile(os.path.join(dir_name, f))] # Create a list of file names with full path, eliminating subdirectory entries


In [3]:
import codecs

#max_files = 1000 #remove to get the entire corpus
raw_corpus = ''
for file in fList1: #[0:max_files] 
    with codecs.open(file, 'r', 'utf-8') as f:
    #with open(file, encoding="utf8") as f:
        raw_corpus += ''.join(f.read())   


In [4]:
import re
corpus = re.sub(r"(\n|\r)+""|(@\w+)+", ' ', raw_corpus) #remove backslashes and words starting with @
#corpus = re.sub(r"(as soon)+" "|(as well)+" "|(as if)+" "|(as quickly as possible)+" "|(as long)" "|(as usual)+" "|(such as)+" "|(as of yet)+" "|(as much)+" "|(as many)+" "|(like that)+" "|(like this)+" "|(like you)+" "|(like me)+" "|(like him)+" "|(like us)+" "|(like her)+" "|(anybody like)+" "|(anyone like)+", "", corpus)
regex_filter = r"(as soon)|(as well)|(as if)|(as \w+ as possible)|(as before)|\
(as long)|(as usual)|(as ever)|\
(such as)|(as of yet)|(as much)|(as many)|\
(like that)|(like this)|(like you)|(like me)|(like him)|(like us)|(like her)|\
(like everything else)|(like everyone else)|(anybody like)|(anyone like)"


In [5]:
re.search(regex_filter, u'as long as') 

**the algorithm:**
#take a big corpus with trite similes (The Daily Mail, http://cs.nyu.edu/~kcho/DMQA/, or a pulp fiction book), prepare clean sentences, traverce throug the corpus sen by sen, POS-tagging each and looking for sentences containing prepositions "like" and "as" (tag = "IN") and exclude "as soon as, as well as, as usual, such as, as of yet, as much, like that, like this.." ((alternatively, use dependency parser to accurately cut out a phrase. but it's a pain and may be an overkill)) Add these sentences to a target corpus. Cut out a simile candidate out of each sentence; optionally: replace "likes" and "ases" with a "comparator". 
#approach 1:
#Use fuzzywazzy to do fuzzy matching of the simile candidates across the corpus. Find those that at least 98% similar and appear multiple times (over 10) across the corpus - those are thrite similes (or common grammatical constuctions containing 'like' or 'as' that we missed during cleaning). Build a corpus of trite similes. With a testing set, repeat all steps up to fuzzywazzy. Then, instead of fuzzy matching candidates across the testinf set, fuzzy match them with the trite similes corpus. Highlight (tag) if a match is found.  
#approach 2: 
#Sort words in each set alphabetically. Then build an n-gram counter (may be plot a histogram) - a dictionary with an n-gram as a key and how many times is appears in the corpus as a value. In a new text, repeat all steps up to the last one and then find new n-grams in the dictionary. If the new n-grams are among the most frequently met n-grams in the corpus, these n-grams constitute trite similes. Then use them as a trite similes corpus and compare to the testing set as described in the approach 1. 


In [6]:
def tokenize_text(corpus, regex_filter):
    sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sents = sent_tokenizer.tokenize(corpus) # Split text into sentences 
    if regex_filter:
        raw_sents = [sent for sent in raw_sents if not re.search(regex_filter, sent)]
    return [nltk.word_tokenize(word) for word in raw_sents]

In [7]:
sentences = tokenize_text(corpus, regex_filter)

In [8]:
sentences[10]

[u'He',
 u'had',
 u'just',
 u'entered',
 u',',
 u'wearing',
 u'an',
 u'embroidered',
 u'court',
 u'uniform',
 u',',
 u'knee',
 u'breeches',
 u',',
 u'and',
 u'shoes',
 u',',
 u'and',
 u'had',
 u'stars',
 u'on',
 u'his',
 u'breast',
 u'and',
 u'a',
 u'serene',
 u'expression',
 u'on',
 u'his',
 u'flat',
 u'face',
 u'.']

In [9]:
comparisons = []
for sent in sentences:
    if not 'like' in sent and not 'as' in sent: 
        continue 
    # exlude a single 'as', leaving in only '...as ... as...'
    if not 'like' in sent and len([word for word in sent if word=='as']) == 1: 
        continue
    pos_tagged = nltk.pos_tag(sent)
    for pair in pos_tagged:
        if pair[1] == 'IN':
            if pair[0] == 'like':
                comparisons.append(sent)
            elif pair[0] == 'as':
                comparisons.append(sent)


In [10]:
comparisons[: ]

[[u'Can',
  u'one',
  u'be',
  u'calm',
  u'in',
  u'times',
  u'like',
  u'these',
  u'if',
  u'one',
  u'has',
  u'any',
  u'feeling',
  u'?',
  u'\u201d',
  u'said',
  u'Anna',
  u'P\xe1vlovna',
  u'.'],
 [u'I',
  u'confess',
  u'all',
  u'these',
  u'festivities',
  u'and',
  u'fireworks',
  u'are',
  u'becoming',
  u'wearisome.\u201d',
  u'\u201cIf',
  u'they',
  u'had',
  u'known',
  u'that',
  u'you',
  u'wished',
  u'it',
  u',',
  u'the',
  u'entertainment',
  u'would',
  u'have',
  u'been',
  u'put',
  u'off',
  u',',
  u'\u201d',
  u'said',
  u'the',
  u'prince',
  u',',
  u'who',
  u',',
  u'like',
  u'a',
  u'wound-up',
  u'clock',
  u',',
  u'by',
  u'force',
  u'of',
  u'habit',
  u'said',
  u'things',
  u'he',
  u'did',
  u'not',
  u'even',
  u'wish',
  u'to',
  u'be',
  u'believed',
  u'.'],
 [u'They',
  u'have',
  u'decided',
  u'that',
  u'Buonaparte',
  u'has',
  u'burnt',
  u'his',
  u'boats',
  u',',
  u'and',
  u'I',
  u'believe',
  u'that',
  u'we',
  u'are',
  

In [43]:

# from stat_parser import Parser
# parser = Parser()
# parser.parse(comparisons[100])

In [44]:
comparisons[0]

[u'Can',
 u'one',
 u'be',
 u'calm',
 u'in',
 u'times',
 u'like',
 u'these',
 u'if',
 u'one',
 u'has',
 u'any',
 u'feeling',
 u'?',
 u'\u201d',
 u'said',
 u'Anna',
 u'P\xe1vlovna',
 u'.']

In [49]:
similes_candidates = []
for sent in comparisons:
    start_index = -1
    words_after = -1
    if 'like' in sent:
        start_index = sent.index('like')
        #two_words_before_like = max(0, index_of_like - 4)
        words_after = min(len(sent), start_index + 6)
    elif 'as' in sent:
        start_index = sent.index('as')
        words_after = min(len(sent), start_index + 8)
        
    if start_index >= 0 and words_after > 0:
        try: 
            index_of_punkt = max(sent.index(','), 0)
            if index_of_punkt > start_index: 
                words_after = min(words_after, index_of_punkt)
        except ValueError: # just ignore
            pass
    similes_candidates.append(sent[start_index:words_after])


In [50]:
similes_candidates[:10]

[[u'like', u'these', u'if', u'one', u'has', u'any'],
 [u'like', u'a', u'wound-up', u'clock', u',', u'by'],
 [u'like', u'an', u'actor', u'repeating', u'a', u'stale'],
 [u'like', u'it', u',', u'she', u'became', u'enthusiastic'],
 [u'like', u'a', u'child', u'in', u'a', u'toyshop'],
 [u'as', u'a', u'specially', u'choice', u'delicacy', u'a', u'piece', u'of'],
 [u'as', u'a', u'specially', u'choice', u'delicacy', u'a', u'piece', u'of'],
 [u'like', u'a', u'well-garnished', u'joint', u'of', u'roast'],
 [u'like', u'his', u'sister\u2019s'],
 [u'like', u'Russia\u2014barbaric', u'as', u'she', u'is', u'said']]

In [51]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
stop_words = set(['a', 'an', 'and', 'or', 'the', \
                  'his', 'her', 'my', 'your', 'their', 'our', \
                  'i', 'you', 'he', 'she', 'it', 'they', 'who', 'that', 'whose', \
                  'is', 'are', 'was', 'will', 'would', \
                  '.',',','-',':',';','!','?'])

def preprocess_words(wordlist): 
    wordset = set([])
    for word in wordlist: 
        word = word.lower()
        if word not in stop_words and len(word) > 1: 
            if word != 'as':
                word = lemmatizer.lemmatize(word)
            if word == 'like' or word == 'as': 
                word = '$cmpr'
            wordset.add(word)
    return wordset 
        

def init_corpus_2match(wordlists): 
    wordsets = []
    for words in wordlists: 
        if not words:
            continue
        wordset = preprocess_words(words)
        if not(not wordset):
            wordsets.append(wordset)
    return wordsets


''' Returns a list of matches for 'phrase' in 'wordsets' with 'min_similarity' 
'''
def fuzzy_match(words_in, corpus, min_similarity): 
    # TODO: must be optimized!!
    phraset = preprocess_words(words_in)
    #print ("Input phraseset is {}".format(phraset))
    nb_input = len(phraset)
    matches = []
    for wordset in corpus: 
        intersect = phraset.intersection(wordset)
        n = len(intersect)
        if n/min(nb_input, len(wordset)) >= min_similarity and not(n < 2 and next(iter(intersect))=='$cmpr'): 
            #print(wordset)
            matches.append(wordset)
    return matches
        





In [52]:
candidates = similes_candidates
corpus_2match = init_corpus_2match(candidates)
covered = set([])
count_dict = {}
for cand in candidates:
    if not cand: 
        continue
    phrase = ' '.join(cand)
    if phrase in covered:
        continue
    covered.add(phrase)
    result = fuzzy_match(cand, corpus_2match, 0.75)
    #print("result is {}".format(result))
    if result:
        count_dict[phrase] = len(result)


In [53]:
import operator
sorted_counts = sorted(count_dict.items(), key=operator.itemgetter(1))
sorted_counts.reverse()
sorted_counts

[(u'like a man .', 39),
 (u'like a child .', 26),
 (u'as a child', 26),
 (u'like a child', 26),
 (u'like it ? \u201d said a', 13),
 (u'like ... . She is like', 13),
 (u'like the others , and like', 11),
 (u'like the others', 11),
 (u'like the prince', 11),
 (u'like a child when the doctor', 10),
 (u'as Nat\xe1sha and I used to as children', 10),
 (u'like children in an embarrassing situation', 10),
 (u'like a son .', 10),
 (u'like a girl ( it was', 9),
 (u'like a child taken out for', 9),
 (u'like children of my own ,', 9),
 (u'like it before .', 9),
 (u'like a child at a dancing', 8),
 (u'like a suffering child and to', 8),
 (u'like a child he made sport', 8),
 (u'like people who', 8),
 (u'like a dream .', 8),
 (u'like a child and quickly shuffling', 8),
 (u'like a child who , holding', 8),
 (u'like a child till tears came', 8),
 (u'like an invalid or a child', 8),
 (u'like a child in a toyshop', 8),
 (u'like a child , he fell', 8),
 (u'as he knew it', 7),
 (u'like a stone , and he', 

In [54]:
from sklearn.externals import joblib
min_freq = 3

top_results_corpus = init_corpus_2match([item[0].split(' ') for item in count_dict.items() if item[1] >= min_freq])

# save 
joblib.dump(count_dict, "top_similes_corpus.v0.pkl")

['top_similes_corpus.v0.pkl']

In [42]:
## simile classification code ##

# similes_corpus = joblib.load("top_similes_corpus.v0.pkl")
# matches = fuzzy_match(new_sent_words, similes_corpus, 0.75)
# if len(matches) >= min_freq
#     print("'{}' is a trite simile".format(new_sent_words))
# else 
#     print("'{}' is NOT a trite simile".format(new_sent_words))



## Backup code 

In [ ]:
# import fuzzywuzzy
# from fuzzywuzzy import fuzz
# from fuzzywuzzy import process


In [ ]:
# choices = []
# for each in similes_candidates:
#     choices.append(" ".join(each))


In [ ]:
# count_dict = {}

# for string in set(choices):
#     result = process.extract(string, choices, limit=1000) #default limit = 5
#     num_matches = 0
#     for each in result:
#         if each[1] > 98:
#             num_matches +=1
#     count_dict[string] = num_matches


In [ ]:
# write 
# from sklearn.externals import joblib
# joblib.dump(count_dict, "count_dict_output.pkl")

In [ ]:
# count_dict = sorted(count_dict.items(), key=operator.itemgetter(1))
# count_dict.reverse()


In [ ]:
#read 
#count_dict_fromfile = joblib.load("count_dict_output.pkl")